In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Урок 3. Классификация. Логистическая регрессия.

**1. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.**

In [6]:
# Пространство признаков. Стаж, цена занятия и квалификация.
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 5, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]])

# Целевая переменная (средний балл ЕГЭ)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

In [7]:
# Стандартизация вектора x
calc_std_fit = lambda x: (x - x.mean()) / x.std()

In [8]:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    err = np.sum(err)
    return err

In [9]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [10]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [14]:
# Стандартизируем пространство признаков.
X_st = X.copy().astype(np.float64)

for i in range(1, X_st.shape[0]):
    X_st[i] = calc_std_fit(X_st[i])

In [12]:
eval_model(X_st, y, iterations=5000, alpha=1e-4)

500 [ 0.49490498 -0.15176383  0.63606273  1.51685213] 0.7537963671147336
1000 [ 0.49302779 -0.16517964  0.62452877  1.51076764] 0.746724940751059
1500 [ 0.49108348 -0.1785107   0.61308755  1.50477718] 0.7397590436228698
2000 [ 0.48907297 -0.19175599  0.60174001  1.49888154] 0.7328987295542733
2500 [ 0.48699728 -0.20491448  0.59048704  1.49308149] 0.7261440408235792
3000 [ 0.48485747 -0.2179851   0.57932959  1.48737779] 0.7194950087719248
3500 [ 0.48265464 -0.2309668   0.56826856  1.48177119] 0.7129516542296922
4000 [ 0.48038998 -0.24385851  0.5573049   1.47626245] 0.7065139877424161
4500 [ 0.4780647  -0.25665911  0.54643953  1.4708523 ] 0.7001820095791962
5000 [ 0.47568011 -0.26936752  0.5356734   1.46554146] 0.6939557095083029


array([ 0.47568011, -0.26936752,  0.5356734 ,  1.46554146])

**2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.**

**3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который 1. уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).**

**4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).**

**5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.**

**6. Могла ли модель переобучиться? Почему?**